<a href="https://colab.research.google.com/github/JYL480/QnAWithContext/blob/main/QnAModelWithContext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
  from datasets import load_dataset
except:
  !pip install datasets
  from datasets import load_dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [ ]:
from datasets import load_dataset

squad = load_dataset("squad", split="train[:12000]")
# We want to start off small first, so that the training will not take too long

In [ ]:
squad = squad.train_test_split(test_size=0.2)

In [ ]:
# We will use the DistillBERT tokenizer!!

from transformers import AutoTokenizer
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# Now we will preprocess the data!!
# This is a very important step!!

def preprocess_function(examples):
  # We will get all the questions first!
  questions = [q.strip() for q in examples["question"]]
  # we have to strip() which removes the leading spaces, this can for the inputs of user as well!

  # You can put sequences or inputs in pair!
  # Note that for Bert Max sequence length/max_position emebding = 512, but here we reduce the max_sequence length to 384
  inputs = tokenizer(
      text=questions,
      text_pair = examples["context"],
      max_length = 384,
      # If the sequence length is to large we will truncate only the context and not the question, hence we use second!!!
      truncation = "only_second",
      # This offset mapping is where to the starting and end index position of char will be reutnr of each words
      return_offsets_mapping=True,
      # Note that when we do batching later, all the batches need to be of equal lenght, hence we will batch it
      # For good utilisation!!
      padding="max_length"

  )

  offset_mapping = inputs.pop("offset_mapping")
  # Here we will get the start and end of each token for the indexes?
  answers = examples["answers"]
  start_positions = []
  end_positions = []

  for i, offset in enumerate(offset_mapping):
    answer = answers[i]
    start_char = answer["answer_start"][0]
    end_char = answer["answer_start"][0] + len(answer["text"][0])
    sequence_ids = inputs.sequence_ids(i)

    # Find the start and end of the context
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    # If the answer is not fully inside the context, label it (0, 0)
    if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # Otherwise it's the start and end token positions
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

  inputs["start_positions"] = start_positions
  inputs["end_positions"] = end_positions
  return inputs

In [ ]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)
tokenized_squad.column_names

Map:   0%|          | 0/9600 [00:00<?, ? examples/s]

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

{'train': ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
 'test': ['input_ids', 'attention_mask', 'start_positions', 'end_positions']}

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained("distilbert/distilbert-base-uncased").to(device)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [ ]:
from torch.utils.data import DataLoader
import os

# Define your DataLoaders with data_collator
train_dataloader = DataLoader(tokenized_squad["train"], batch_size=16, num_workers=os.cpu_count(), collate_fn=data_collator)
test_dataloader = DataLoader(tokenized_squad["test"], batch_size=16, num_workers=os.cpu_count(), collate_fn=data_collator)

batch = next(iter(train_dataloader))
batch['input_ids'].shape

torch.Size([16, 384])

In [ ]:

optimizer = torch.optim.AdamW(params=model.parameters(),
                            lr = 0.00002)

from tqdm.auto import tqdm

num_epochs = 2
model.train()

for epoch in tqdm(range(num_epochs)):
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask,
                        start_positions=start_positions, end_positions=end_positions)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        print(f"Epoch: {epoch}, Loss: {loss}")

  0%|          | 0/2 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch: 0, Loss: 0.3981962203979492
Epoch: 0, Loss: 0.5215339660644531
Epoch: 0, Loss: 0.8192795515060425
Epoch: 0, Loss: 0.635993242263794
Epoch: 0, Loss: 0.6509624719619751
Epoch: 0, Loss: 0.8679591417312622
Epoch: 0, Loss: 0.6041713356971741
Epoch: 0, Loss: 0.7814739942550659
Epoch: 0, Loss: 0.6852315664291382
Epoch: 0, Loss: 0.8073239326477051
Epoch: 0, Loss: 1.4550211429595947
Epoch: 0, Loss: 1.0118091106414795
Epoch: 0, Loss: 0.9056591987609863
Epoch: 0, Loss: 0.5628124475479126
Epoch: 0, Loss: 1.142425775527954
Epoch: 0, Loss: 1.4371216297149658
Epoch: 0, Loss: 1.254491925239563
Epoch: 0, Loss: 0.7094515562057495
Epoch: 0, Loss: 1.1595383882522583
Epoch: 0, Loss: 1.258088231086731
Epoch: 0, Loss: 1.3447024822235107
Epoch: 0, Loss: 1.1596434116363525
Epoch: 0, Loss: 1.1916159391403198
Epoch: 0, Loss: 2.046754837036133
Epoch: 0, Loss: 0.8906792402267456
Epoch: 0, Loss: 1.9761831760406494
Epoch: 0, Loss: 1.4397974014282227
Epoch: 0, Loss: 1.2991019487380981
Epoch: 0, Loss: 1.1994888

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch: 0, Loss: 1.1504160165786743
Epoch: 1, Loss: 0.28267714381217957
Epoch: 1, Loss: 0.5293251872062683
Epoch: 1, Loss: 0.6456435322761536
Epoch: 1, Loss: 0.5452819466590881
Epoch: 1, Loss: 0.6217437982559204
Epoch: 1, Loss: 0.847220778465271
Epoch: 1, Loss: 0.41523271799087524
Epoch: 1, Loss: 0.6180839538574219
Epoch: 1, Loss: 0.5032660365104675
Epoch: 1, Loss: 0.6253949999809265
Epoch: 1, Loss: 0.8090202808380127
Epoch: 1, Loss: 0.6646323204040527
Epoch: 1, Loss: 0.4742577075958252
Epoch: 1, Loss: 0.872505784034729
Epoch: 1, Loss: 0.8901493549346924
Epoch: 1, Loss: 1.160179615020752
Epoch: 1, Loss: 0.8597832918167114
Epoch: 1, Loss: 0.4827992916107178
Epoch: 1, Loss: 0.9075161814689636
Epoch: 1, Loss: 1.1128194332122803
Epoch: 1, Loss: 0.7231180667877197
Epoch: 1, Loss: 0.7020272016525269
Epoch: 1, Loss: 0.8082300424575806
Epoch: 1, Loss: 1.3104478120803833
Epoch: 1, Loss: 0.2531864047050476
Epoch: 1, Loss: 1.1032071113586426
Epoch: 1, Loss: 1.0339431762695312
Epoch: 1, Loss: 0.861

In [ ]:
def prediction(question, context):
  inputs = tokenizer(question, context, return_tensors="pt")
  input_ids = inputs["input_ids"].to(device)
  attention_mask = inputs["attention_mask"].to(device)
  outputs = model(input_ids=input_ids, attention_mask=attention_mask)
  start_logits = outputs.start_logits
  end_logits = outputs.end_logits
  start_index = torch.argmax(start_logits)
  end_index = torch.argmax(end_logits)
  answer = tokenizer.decode(input_ids[0][start_index:end_index+1])
  return answer



In [ ]:
question = "Who painted the Mona Lisa?"
context = "The Mona Lisa, painted by Leonardo da Vinci, is one of the most famous artworks in the world, known for its enigmatic smile"

In [ ]:
print(prediction(question, context))

leonardo da vinci
